In [144]:
from pyspark.sql import SparkSession

# SparkSession 객체 생성
spark = SparkSession.builder \
    .appName("stock") \
    .getOrCreate()



# pandas 버전
# pandas_df = pd.read_csv('titanic_train.csv', header='infer')

#spark.read.csv() 메소드를 이용하여 csv 파일을 로드하고 DataFrame으로 변환. 
# CSV 파일을 pandas 데이터프레임으로 불러올 때, 'NULL' 문자열을 NaN 값으로 대체하도록 설정
stock_sdf = spark.read.csv('./data/stock_info_0315.csv', header=True, inferSchema=True,  nullValue='NULL')
print('stock sdf type:', type(stock_sdf))

stock_sdf.show(5)

# spark DataFrame을 메모리에 cache
stock_sdf = stock_sdf.cache()



# pandas 데이터 불러오기 
import pandas as pd

na_values = ['NULL']
stock_pdf = pd.read_csv('./data/stock_info_0315.csv', header='infer', na_values=na_values)


stock sdf type: <class 'pyspark.sql.dataframe.DataFrame'>
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|종목코드|      종목명| 종가|등락률|    시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|  거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|  000020|    동화약품| 9350| -0.85|261159244500|2024-03-15| 736.0| 12.7|   null|   null| 13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02| 9450| 9450| 9300| 7220

### Spark DataFrame의 withColumn() 메소드 알아보기
* pandas DataFrame은 [] 을 이용하여 기존 컬럼값을 update, 또는 신규 컬럼 추가, 묵시적으로 컬럼 타입을 변경할 수 있음. 컬럼명 변경시는 rename()을 사용. 명시적인 컬럼 타입 변경은 astype()적용
* spark DataFrame은 withColumn() 메소드를 이용하여 기존 컬럼값을 update, 컬럼 타입 변경, 신규 컬럼값을 추가할 수 있음. 
* withColumn('신규 또는 Update되는 컬럼명', '신규 또는 update되는 값')을 인자로 가짐. 
* 신규 또는 update되는 값을 생성 시에 기존 컬럼을 기반으로 한다면 신규 컬럼은 문자열로, 기존 컬럼은 반드시 컬럼형(col('컬럼명'))을 이용하여 적용.
* 신규 컬럼값을 추가하는 것은 select() 메소드로도 가능
* 컬럼명을 변경하는 것은 withColumnRename() 메소드로 수행.

In [131]:
stock_pdf.info()
stock_pdf.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   종목코드      2671 non-null   object 
 1   종목명       2671 non-null   object 
 2   종가        2671 non-null   int64  
 3   등락률       2671 non-null   float64
 4   시가총액      2671 non-null   int64  
 5   기준일       2671 non-null   object 
 6   eps       1600 non-null   float64
 7   per       1600 non-null   float64
 8   선행eps     511 non-null    float64
 9   선행per     511 non-null    float64
 10  bps       2381 non-null   float64
 11  pbr       2381 non-null   float64
 12  주당배당금     2622 non-null   float64
 13  배당수익률     2622 non-null   float64
 14  외국인보유수량   2671 non-null   int64  
 15  외국인지분율    2671 non-null   float64
 16  외국인한도수량   2671 non-null   int64  
 17  외국인한도소진율  2671 non-null   float64
 18  시가        2671 non-null   int64  
 19  고가        2671 non-null   int64  
 20  저가        2671 non-null   int6

,종목코드,종목명,종가,등락률,시가총액,기준일,eps,per,선행eps,선행per,...,시가,고가,저가,거래량,거래대금,기업고유번호,시장구분,종목구분,섹터,업종명
0,000020,동화약품,9350,-0.85,261159244500,2024-03-15,736.0,12.70,NaN,NaN,...,9450,9450,9300,72206,674278570,119195.0,KOSPI,보통주,건강관리,의약품
1,000040,KR모터스,465,0.00,44704386225,2024-03-15,NaN,NaN,NaN,NaN,...,0,0,0,0,0,112378.0,KOSPI,보통주,경기관련소비재,운수장비
2,000050,경방,8470,-1.63,232207336900,2024-03-15,177.0,47.85,NaN,NaN,...,8610,8690,8270,11024,93485040,101628.0,KOSPI,보통주,경기관련소비재,유통업


In [134]:
import numpy as np

stock_pdf_copied = stock_pdf.copy()
# Pandas DataFrame 신규 컬럼 추가
stock_pdf_copied['Extra_등락률'] = stock_pdf_copied['등락률'] * 100
# 기존 컬럼 update
stock_pdf_copied['종가'] = stock_pdf_copied['종가'] + 1000
# 기존 컬럼의 Data Type 변경.  
stock_pdf_copied['등락률'] = stock_pdf_copied['등락률'].astype(np.int64)

stock_pdf_copied.info()
stock_pdf_copied[['종목명', 'Extra_등락률', '종가', '등락률']].head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 29 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   종목코드       2671 non-null   object 
 1   종목명        2671 non-null   object 
 2   종가         2671 non-null   int64  
 3   등락률        2671 non-null   int64  
 4   시가총액       2671 non-null   int64  
 5   기준일        2671 non-null   object 
 6   eps        1600 non-null   float64
 7   per        1600 non-null   float64
 8   선행eps      511 non-null    float64
 9   선행per      511 non-null    float64
 10  bps        2381 non-null   float64
 11  pbr        2381 non-null   float64
 12  주당배당금      2622 non-null   float64
 13  배당수익률      2622 non-null   float64
 14  외국인보유수량    2671 non-null   int64  
 15  외국인지분율     2671 non-null   float64
 16  외국인한도수량    2671 non-null   int64  
 17  외국인한도소진율   2671 non-null   float64
 18  시가         2671 non-null   int64  
 19  고가         2671 non-null   int64  
 20  저가      

,종목명,Extra_등락률,종가,등락률
0,동화약품,-85.0,10350,0
1,KR모터스,0.0,1465,0
2,경방,-163.0,9470,-1


In [136]:
from pyspark.sql.functions import col

stock_sdf_copied = stock_sdf.select('*') # stock_sdf 를 copy


# withColumn('신규 또는 Update되는 컬럼명', '신규 또는 update되는 값')을 인자로 가짐. 
# 신규 또는 update되는 값을 생성 시에 기존 컬럼을 기반으로 한다면 신규 컬럼은 문자열로, 기존 컬럼은 반드시 컬럼형(col('컬럼명'))을 이용하여 적용
# 신규 컬럼 Extra_등락률을 기존 컬럼 등락률을 이용하여 추가. 신규 컬럼은 'Extra_등락률' 문자열로, 기존 컬럼은 col('등락률')로 적용. 
stock_sdf_copied = stock_sdf_copied.withColumn('Extra_등락률', col('등락률') * 100) 

# 기존 컬럼 종가 값을 update
stock_sdf_copied = stock_sdf_copied.withColumn('종가', col('종가') + 1000)

# 기존 컬럼 등락률의 데이터 타입을 Integer로 변경. 
stock_sdf_copied = stock_sdf_copied.withColumn('등락률', col('등락률').cast('Integer'))

stock_sdf_copied.printSchema()
stock_sdf_copied.limit(3).show()

root
 |-- 종목코드: string (nullable = true)
 |-- 종목명: string (nullable = true)
 |-- 종가: integer (nullable = true)
 |-- 등락률: integer (nullable = true)
 |-- 시가총액: long (nullable = true)
 |-- 기준일: date (nullable = true)
 |-- eps: double (nullable = true)
 |-- per: double (nullable = true)
 |-- 선행eps: double (nullable = true)
 |-- 선행per: double (nullable = true)
 |-- bps: double (nullable = true)
 |-- pbr: double (nullable = true)
 |-- 주당배당금: double (nullable = true)
 |-- 배당수익률: double (nullable = true)
 |-- 외국인보유수량: long (nullable = true)
 |-- 외국인지분율: double (nullable = true)
 |-- 외국인한도수량: long (nullable = true)
 |-- 외국인한도소진율: double (nullable = true)
 |-- 시가: integer (nullable = true)
 |-- 고가: integer (nullable = true)
 |-- 저가: integer (nullable = true)
 |-- 거래량: integer (nullable = true)
 |-- 거래대금: long (nullable = true)
 |-- 기업고유번호: integer (nullable = true)
 |-- 시장구분: string (nullable = true)
 |-- 종목구분: string (nullable = true)
 |-- 섹터: string (nullable = true)
 |-- 업종명: string (nullable

In [137]:
# 상수 값으로 update 시에 아래와 같이 수행하면 오류가 발생. 반드시 update할 값은 컬럼형이 되어야 함. 
stock_sdf_copied = stock_sdf_copied.withColumn('Extra_등락률', 10)

PySparkTypeError: [NOT_COLUMN] Argument `col` should be a Column, got int.

In [138]:
from pyspark.sql.functions import lit

# 상수 값으로 update 시 반드시 lit() 함수를 적용하여야 함. 
stock_sdf_copied = stock_sdf_copied.withColumn('Extra_values', lit(10))

# 상수 값으로 신규 컬럼 생성시에도 반드시 lit() 함수를 적용해야 함. 
stock_sdf_copied = stock_sdf_copied.withColumn('New_Name', lit('Test_name'))

stock_sdf_copied.limit(3).show()

+--------+--------+-----+------+------------+----------+-----+-----+-------+-------+-------+----+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+---------+------------+--------+--------+--------------+--------+------------+------------+---------+
|종목코드|  종목명| 종가|등락률|    시가총액|    기준일|  eps|  per|선행eps|선행per|    bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율|시가|고가|저가|거래량| 거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|Extra_등락률|Extra_values| New_Name|
+--------+--------+-----+------+------------+----------+-----+-----+-------+-------+-------+----+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+---------+------------+--------+--------+--------------+--------+------------+------------+---------+
|  000020|동화약품|10350|     0|261159244500|2024-03-15|736.0| 12.7|   null|   null|13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02|9450|9450|93

In [139]:
from pyspark.sql.functions import col, substring

# select() 메소드를 이용하여 컬럼 추가. SQL substring() 함수를 이용하여 문자열의 일부를 추출하여 신규 컬럼 생성. 

# select a.*, 종가 as closing_price from stock_sdf a
stock_sdf_copied = stock_sdf_copied.select('*', col('종가').alias('closing_price'))

# select a.*, substring('종목구분', 0, 1) as stock_division from stock_sdf a
stock_sdf_copied = stock_sdf_copied.select('*', substring('종목구분', 0, 2).alias('stock_division')) 


stock_sdf_copied.limit(3).show()

+--------+--------+-----+------+------------+----------+-----+-----+-------+-------+-------+----+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+---------+------------+--------+--------+--------------+--------+------------+------------+---------+-------------+--------------+
|종목코드|  종목명| 종가|등락률|    시가총액|    기준일|  eps|  per|선행eps|선행per|    bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율|시가|고가|저가|거래량| 거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|Extra_등락률|Extra_values| New_Name|closing_price|stock_division|
+--------+--------+-----+------+------------+----------+-----+-----+-------+-------+-------+----+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+---------+------------+--------+--------+--------------+--------+------------+------------+---------+-------------+--------------+
|  000020|동화약품|10350|     0|261159244500|2024-03-15|736.0| 12.7|   null|   null|13165.0|0.71|     18

In [140]:
from pyspark.sql.functions import split

# SQL function split()을 이용하여 문자열을 ','로 분리하여 새로운 컬럼명 Name1, Name2 생성. 
# O 문자를 기준으로 split하여 첫번째 요소 반환 
stock_sdf_copied = stock_sdf_copied.withColumn('Name1', split(col('시장구분'), 'O').getItem(0))
# O 문자를 기준으로 split하여 두번째 요소 반환 
stock_sdf_copied  = stock_sdf_copied.withColumn('Name2', split(col('시장구분'), 'O').getItem(1))


stock_sdf_copied.select("Name1", "Name2").limit(10).show()


+-----+-----+
|Name1|Name2|
+-----+-----+
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
|    K|  SPI|
+-----+-----+



In [142]:
# withColumnRenamed('기존 컬럼명', '변경 컬럼명')으로 컬럼명 변경. 
stock_sdf_copied = stock_sdf_copied.withColumnRenamed('시장구분', 'market_division')
stock_sdf_copied = stock_sdf_copied.withColumnRenamed('종목코드', 'stock_code')

#변경하려는 컬럼명이 없어도 오류를 발생 시키지 않음. 유의 필요. 
stock_sdf_copied = stock_sdf_copied.withColumnRenamed('시장 구분_X', 'market_division_X')

stock_sdf_copied.select('market_division', 'stock_code', col('종목명').alias('stock_name')).limit(10).show()

+---------------+----------+--------------+
|market_division|stock_code|    stock_name|
+---------------+----------+--------------+
|          KOSPI|    000020|      동화약품|
|          KOSPI|    000040|      KR모터스|
|          KOSPI|    000050|          경방|
|          KOSPI|    000070|    삼양홀딩스|
|          KOSPI|    000075|  삼양홀딩스우|
|          KOSPI|    000080|    하이트진로|
|          KOSPI|    000087|하이트진로2우B|
|          KOSPI|    000100|      유한양행|
|          KOSPI|    000105|    유한양행우|
|          KOSPI|    000120|    CJ대한통운|
+---------------+----------+--------------+



### Spark DataFrame의 컬럼 삭제와 로우(레코드) 삭제
* Pandas DataFrame은 drop() 메소드의 axis를 기반으로 컬럼(axis=1) 또는 로우(axis=0)를 삭제할 수 있으나
* Spark DataFrame drop() 메소드는 컬럼 삭제만 가능. 단일/여러개의 컬럼을 삭제 할 수 있음. 단 여러개의 컬럼 삭제 시 list로 입력 할 수 없으며 개별 컬럼명들이 입력되어야 함. 
* Spark DataFrame은 기본적으로는 특정 조건에 따른 로우 삭제가 어려움. 로우 삭제 대신 filter() 메소드를 이용하여 해당 조건의 데이터를 다시 만들어냄. 
* Pandas의 None 값을 Null을 의미하여 Spark에서는 null로 변환됨. 
* 값이 있는 record는 dropna() 메소드 또는 DataFrame.na.drop()을 이용하여 삭제 할 수 있음. 또는 filter() 조건에서 Not null조건으로 다시 만들어 냄.
* DataFrame.na는 DataFrameNaFunctions 객체임.

In [146]:
stock_pdf_dropped = stock_pdf.drop('종목명', axis=1, inplace=False)
stock_pdf_dropped.columns

Index(['종목코드', '종가', '등락률', '시가총액', '기준일', 'eps', 'per', '선행eps', '선행per',
       'bps', 'pbr', '주당배당금', '배당수익률', '외국인보유수량', '외국인지분율', '외국인한도수량',
       '외국인한도소진율', '시가', '고가', '저가', '거래량', '거래대금', '기업고유번호', '시장구분', '종목구분',
       '섹터', '업종명'],
      dtype='object')

In [147]:
from pyspark.sql.functions import col

stock_sdf_copied = stock_sdf.select('*')

# 단일 컬럼 삭제. drop() 메소드 인자로 단일 컬럼명 문자열, 또는 컬럼명 컬럼형을 입력. 
stock_sdf_copied = stock_sdf_copied.drop('종목명')
# drop 컬럼이 존재하지 않아도 오류가 발생하지 않음. 유의 필요. 
stock_sdf_copied = stock_sdf_copied.drop(col('종목명_ㅌ'))

stock_sdf_copied.columns

['종목코드',
 '종가',
 '등락률',
 '시가총액',
 '기준일',
 'eps',
 'per',
 '선행eps',
 '선행per',
 'bps',
 'pbr',
 '주당배당금',
 '배당수익률',
 '외국인보유수량',
 '외국인지분율',
 '외국인한도수량',
 '외국인한도소진율',
 '시가',
 '고가',
 '저가',
 '거래량',
 '거래대금',
 '기업고유번호',
 '시장구분',
 '종목구분',
 '섹터',
 '업종명']

In [148]:
from pyspark.sql.functions import col

#여러개의 컬럼을 삭제할 시 list가 아니라 단일 컬럼명들을 각각 인자로 넣어 주어야 함. 
stock_sdf_copied.drop('종목코드', '종가').limit(3).columns

['등락률',
 '시가총액',
 '기준일',
 'eps',
 'per',
 '선행eps',
 '선행per',
 'bps',
 'pbr',
 '주당배당금',
 '배당수익률',
 '외국인보유수량',
 '외국인지분율',
 '외국인한도수량',
 '외국인한도소진율',
 '시가',
 '고가',
 '저가',
 '거래량',
 '거래대금',
 '기업고유번호',
 '시장구분',
 '종목구분',
 '섹터',
 '업종명']

In [150]:
# 아래는 오류 발생. 여러개의 컬럼들을 삭제 시 컬럼형 인자는 안됨(3.2 버전에서는 오류가 나지만, 3.4 버전에서는 정상 출력)
stock_sdf_copied.drop(col('종목코드'), col('종가')).columns

['등락률',
 '시가총액',
 '기준일',
 'eps',
 'per',
 '선행eps',
 '선행per',
 'bps',
 'pbr',
 '주당배당금',
 '배당수익률',
 '외국인보유수량',
 '외국인지분율',
 '외국인한도수량',
 '외국인한도소진율',
 '시가',
 '고가',
 '저가',
 '거래량',
 '거래대금',
 '기업고유번호',
 '시장구분',
 '종목구분',
 '섹터',
 '업종명']

In [27]:
drop_columns = ['종목코드', '종가']
print(*drop_columns)

stock_code closing_price


In [151]:
drop_columns = ['종목코드', '종가']
drop_columns_col = [col('종목코드'), col('종가')]

stock_sdf_copied.drop(*drop_columns).limit(10).show()

stock_sdf_copied.drop(*drop_columns_col).limit(10).show()

+------+-------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+------+------+------+------+-----------+------------+--------+--------+--------------+----------+
|등락률|     시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율|  시가|  고가|  저가|거래량|   거래대금|기업고유번호|시장구분|종목구분|          섹터|    업종명|
+------+-------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+------+------+------+------+-----------+------------+--------+--------+--------------+----------+
| -0.85| 261159244500|2024-03-15| 736.0| 12.7|   null|   null| 13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02|  9450|  9450|  9300| 72206|  674278570|      119195|   KOSPI|  보통주|      건강관리|    의약품|
|   0.0|  44704386225|2024-03-15|  null| null|   null|   null|   345.0

In [30]:
stock_sdf_copied.dtypes

[('stock_name', 'string'),
 ('closing_price', 'bigint'),
 ('price_change', 'double'),
 ('market_cap', 'bigint'),
 ('base_date', 'string'),
 ('eps', 'double'),
 ('per', 'double'),
 ('leading_eps', 'double'),
 ('leading_per', 'double'),
 ('bps', 'double'),
 ('pbr', 'double'),
 ('dividend_per_share', 'double'),
 ('dividend_yield', 'double'),
 ('foreign_ownership_quantity', 'bigint'),
 ('foreign_ownership_ratio', 'double'),
 ('foreign_limit_quantity', 'bigint'),
 ('foreign_limit_exhaustion_ratio', 'double'),
 ('opening_price', 'bigint'),
 ('high_price', 'bigint'),
 ('low_price', 'bigint'),
 ('trading_volume', 'bigint'),
 ('trading_value', 'bigint'),
 ('company_id', 'double'),
 ('market_division', 'string'),
 ('stock_division', 'string'),
 ('sector', 'string'),
 ('industry_name', 'string')]

In [31]:
# 아래와 같이 logic으로 조건에 맞는 여러개의 컬럼들을 삭제할 수 있음. 
drop_string_columns = [ column_name for column_name, column_type in stock_sdf_copied.dtypes if column_type == 'string']
print('drop 컬럼명:', drop_string_columns)
stock_sdf_copied.drop(*drop_string_columns).limit(10).show()


drop 컬럼명: ['stock_name', 'base_date', 'market_division', 'stock_division', 'sector', 'industry_name']
+-------------+------------+-------------+------+-----+-----------+-----------+--------+----+------------------+--------------+--------------------------+-----------------------+----------------------+------------------------------+-------------+----------+---------+--------------+-------------+----------+
|closing_price|price_change|   market_cap|   eps|  per|leading_eps|leading_per|     bps| pbr|dividend_per_share|dividend_yield|foreign_ownership_quantity|foreign_ownership_ratio|foreign_limit_quantity|foreign_limit_exhaustion_ratio|opening_price|high_price|low_price|trading_volume|trading_value|company_id|
+-------------+------------+-------------+------+-----+-----------+-----------+--------+----+------------------+--------------+--------------------------+-----------------------+----------------------+------------------------------+-------------+----------+---------+--------------+

In [153]:
# Spark DataFrame은 특정 조건으로 로우를 삭제하기가 어려우므로 filter()로 특정 조건에 해당하지 않는 로우를 걸러내는 방식을 적용. 
stock_sdf_removed_market_division_kospi = stock_sdf.filter(col('시장구분') != 'KOSPI')
stock_sdf_removed_market_division_kospi.select('종목명', '섹터', '시장구분').show(3)

+------------+--------+--------+
|      종목명|    섹터|시장구분|
+------------+--------+--------+
|  삼천당제약|건강관리|  KOSDAQ|
|중앙에너비스|  에너지|  KOSDAQ|
|    신라섬유|      IT|  KOSDAQ|
+------------+--------+--------+
only showing top 3 rows



In [160]:
# 하나라도 null또는 nan 값이 있으면 삭제한 결과 DataFrame을 반환. 
print('stock_pdf count:', stock_pdf.shape[0])

# DataFrame의 dropna()메소드는 레코드에 하나라도 null또는 nan 값이 있으면 삭제한 결과 DataFrame을 반환. 
stock_pdf_dropna_01 = stock_pdf.dropna()
print('dropna()적용 후 count:', stock_pdf_dropna_01.shape[0])

stock_pdf count: 2671
dropna()적용 후 count: 395


In [154]:
from pyspark.sql.functions import isnan

print('stock_sdf count:', stock_sdf.count())

# DataFrame의 dropna()메소드는 레코드에 하나라도 null또는 nan 값이 있으면 삭제한 결과 DataFrame을 반환. 
stock_sdf_dropna_01 = stock_sdf.dropna()
print('dropna()적용 후 count:', stock_sdf_dropna_01.count())

stock_sdf count: 2671
dropna()적용 후 count: 395


In [161]:
stock_sdf_dropna_02 = stock_sdf.na.drop()
print('DataFrame.na.drop()적용 후 count:', stock_sdf_dropna_02.count())
print('stock_sdf.na 타입:', type(stock_sdf.na))

DataFrame.na.drop()적용 후 count: 395
stock_sdf.na 타입: <class 'pyspark.sql.dataframe.DataFrameNaFunctions'>


In [162]:
# 특정 컬럼에 Null 이 있는 경우에만 삭제할 경우
stock_sdf_dropna_03 = stock_sdf.na.drop(subset=["per", "eps"]) 
print('DataFrame.na.drop()을 per와 eps 컬럼 적용 후 count:', stock_sdf_dropna_03.count())

DataFrame.na.drop()을 per와 eps 컬럼 적용 후 count: 1600


### Pandas와 Spark에서의 None, Null, NaN 의 구분 - 1
* Python은 None이라는 값이 없는 내장 상수가 있음. None 객체라고도 부리면 이는 NoneType 클래스임. 
* SQL은 원론적으로 None이 아니라 Null 임. 
* numpy는 python None은 처리하기 위해 object 형으로 None을 할당할수 있고, float 형으로 NaN을 할당 할 수 있음. 
* NaN은 원래 Not a Number라는 의미임. 숫자형 array에 값이 없을 경우에는 NaN을 할당함. 
* pandas는 csv와 같은 파일에서 로드 시 특정 컬럼에 데이터가 없을 경우에 문자열 컬럼일 경우 None으로 숫자형 컬럼일 경우 NaN으로 할당. 단 NaN 으로 할당 시에는 int형 컬럼이라도 float형으로 변경됨. 
* Spark는 csv와 같은 파일에서 로드 시 모든 컬럼을 다 Null로 변환. 기본적으로 None은 Null에 할당. 이는 SQL사상과 동일. 
* 하지만 Spark는 pandas DataFrame의 NaN 처리와 어느정도 호환성을 유지하기 위해 NaN도 함께 지원.
* 과거 버전 Spark(Spark 3.0 이하)는 pandas DataFrame을 spark로 변환 시에 NaN 값을 동일하게 NaN으로 변환하였으나 현재는 null로 변환함. 하지만 NaN 값을 명확하게 지정하여 spark DataFrame을 만들 수 있음.
* 결론적으로 NaN은 고려하지 않고 Null만 고려할 수 있도록 Spark DataFrame을 만드는 것이 중요. isnan()은 사용하지 않고, isNull()만 사용할 수 있도록 유도.

In [107]:
val = None
print(type(val), val)

<class 'NoneType'> None


In [117]:
import pandas as pd
import numpy as np

# None은 object형 array에만 넣을 수 있음. 
array = np.array([0, 1, 2, None])
print(array)

# None을 숫자형(np.int는 안되고 np.float)로 입력할 경우는 NaN으로 입력됨. 
array = np.array([0, 1, 2, None], dtype=np.float64)
print(array)

array = np.array([0, 1, 2, np.NaN], dtype=np.float64)
print(array)

[0 1 2 None]
[ 0.  1.  2. nan]
[ 0.  1.  2. nan]


In [166]:
# Spark 3.2 에서는 pandas DataFrame의 NaN을 spark DataFrame으로 변환 시 Null로 변환. spark 3.0 이하 버전에서는 NaN 으로 변환
# Spark 3.4 버전에서는 NaN 으로 변환(현재 사용 버전)
pdf = pd.DataFrame({
    "x": [1, np.NaN], "y": [None, "foo"]
})
sdf = spark.createDataFrame(pdf)

sdf.show()

+---+----+
|  x|   y|
+---+----+
|1.0|null|
|NaN| foo|
+---+----+



In [170]:
# 직접 NaN값을 지정하여 입력할 경우 Spark DataFrame에 NaN 입력 가능. 
sdf = spark.createDataFrame([(1.0, None), (float('nan'), 'foo')], ("x", "y"))
sdf.show()

+---+----+
|  x|   y|
+---+----+
|1.0|null|
|NaN| foo|
+---+----+



### Spark DataFrame에서 Null과 NaN 찾기
* pandas DataFrame의 isnull()과 isna()는 서로 동일한 메소드임. isnull(), isna() 모두 None과 NaN을 모두 찾음. 
* spark DataFrame isNull()은 null만 찾아줌, isnan()은 NaN만 찾음. 또한 isNull()은 컬럼 조건에 붙어서 filter()메소드와 함께 사용되며, isnan()은 pyspark.sql.functions의 함수로 정의됨.
* spark DataFrame의 dropna() 메소드는 NaN과 Null 모두를 찾아서 삭제해줌.
* Not Null 조건으로 찾을 때는 isNotNull() 적용.

In [173]:
print(stock_pdf[['종목명', 'per']].head(10))
print('### isna() 적용 결과 ### ')
print(stock_pdf[['종목명', 'per']].isna().head(10))

print('### isnull() 적용 결과 ### ')
print(stock_pdf[['종목명', 'per']].isnull().head(10))

        종목명    per
0      동화약품  12.70
1     KR모터스    NaN
2        경방  47.85
3     삼양홀딩스   7.77
4    삼양홀딩스우    NaN
5     하이트진로  16.24
6  하이트진로2우B    NaN
7      유한양행  58.18
8     유한양행우    NaN
9    CJ대한통운  15.21
### isna() 적용 결과 ### 
     종목명    per
0  False  False
1  False   True
2  False  False
3  False  False
4  False   True
5  False  False
6  False   True
7  False  False
8  False   True
9  False  False
### isnull() 적용 결과 ### 
     종목명    per
0  False  False
1  False   True
2  False  False
3  False  False
4  False   True
5  False  False
6  False   True
7  False  False
8  False   True
9  False  False


In [177]:
from pyspark.sql.functions import col, isnan

#isNull()은 컬럼 조건에 붙어서 filter()메소드와 함께 사용. isnan()은 pyspark.sql.functions의 함수로 사용. 
print('##### isNull() 적용 결과 #####')
# select * from stock_sdf where per is Null
stock_sdf.filter(col('per').isNull()).show(3) 
#stock_sdf.filter('per is Null').show(10)

print('##### isnan() 함수 적용 결과 #####')
stock_sdf.where(isnan(col('per'))).show(3)

##### isNull() 적용 결과 #####
+--------+--------------+-----+------+-----------+----------+----+----+-------+-------+-----+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+--------+------------+--------+--------+--------------+--------+
|종목코드|        종목명| 종가|등락률|   시가총액|    기준일| eps| per|선행eps|선행per|  bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+--------------+-----+------+-----------+----------+----+----+-------+-------+-----+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+--------+------------+--------+--------+--------------+--------+
|  000040|      KR모터스|  465|   0.0|44704386225|2024-03-15|null|null|   null|   null|345.0|1.35|       0.0|       0.0|      43963412|       45.73|      96138465|           45.73|    0|    0|    0|     0|       0|      112378|   KOSPI|  보통주|경기관련소비재|운수장비|
|  

In [178]:
# spark DataFrame의 dropna()와 DataFrame.na.drop()은 Null 또는 NaN 모두를 찾아서 삭제해줌. 
sdf = spark.createDataFrame([(1.0, None), (float('nan'), 'foo')], ("x", "y"))
print(sdf.show())
sdf.dropna().show()
sdf.na.drop().show()

+---+----+
|  x|   y|
+---+----+
|1.0|null|
|NaN| foo|
+---+----+

None
+---+---+
|  x|  y|
+---+---+
+---+---+

+---+---+
|  x|  y|
+---+---+
+---+---+



In [179]:
stock_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   종목코드      2671 non-null   object 
 1   종목명       2671 non-null   object 
 2   종가        2671 non-null   int64  
 3   등락률       2671 non-null   float64
 4   시가총액      2671 non-null   int64  
 5   기준일       2671 non-null   object 
 6   eps       1600 non-null   float64
 7   per       1600 non-null   float64
 8   선행eps     511 non-null    float64
 9   선행per     511 non-null    float64
 10  bps       2381 non-null   float64
 11  pbr       2381 non-null   float64
 12  주당배당금     2622 non-null   float64
 13  배당수익률     2622 non-null   float64
 14  외국인보유수량   2671 non-null   int64  
 15  외국인지분율    2671 non-null   float64
 16  외국인한도수량   2671 non-null   int64  
 17  외국인한도소진율  2671 non-null   float64
 18  시가        2671 non-null   int64  
 19  고가        2671 non-null   int64  
 20  저가        2671 non-null   int6

#### Null이 있는 컬럼명과 Null 건수를 찾기

In [188]:
from pyspark.sql.functions import count, when

# select count(case when 종목명 is null then 종목명), count(case when 종가 is null then 종가),,,,, from stock_sdf
stock_sdf.select([count(when (col(c).isNull(), c)).alias(c) for c in stock_sdf.columns]).show()


# stock_sdf.select([count(when(isnan(c), c)).alias(c) for c in stock_sdf.columns])

+--------+------+----+------+--------+------+----+----+-------+-------+---+---+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+--------+------------+--------+--------+----+------+
|종목코드|종목명|종가|등락률|시가총액|기준일| eps| per|선행eps|선행per|bps|pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율|시가|고가|저가|거래량|거래대금|기업고유번호|시장구분|종목구분|섹터|업종명|
+--------+------+----+------+--------+------+----+----+-------+-------+---+---+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+--------+------------+--------+--------+----+------+
|       0|     0|   0|     0|       0|     0|1071|1071|   2160|   2160|290|290|        49|        49|             0|           0|             0|               0|   0|   0|   0|     0|       0|         117|       0|       0| 395|     0|
+--------+------+----+------+--------+------+----+----+-------+-------+---+---+----------+----------+--------------+------------+--------------+-----

In [189]:
# select * from stock_sdf where per is not null
stock_sdf.filter(col('per').isNotNull()).select('종목명', 'per', 'eps').show(10) 

+----------------+------+------+
|          종목명|   per|   eps|
+----------------+------+------+
|        동화약품|  12.7| 736.0|
|            경방| 47.85| 177.0|
|      삼양홀딩스|  7.77|9173.0|
|      하이트진로| 16.24|1250.0|
|        유한양행| 58.18|1272.0|
|      CJ대한통운| 15.21|8190.0|
|하이트진로홀딩스|  6.31|1427.0|
|              DL| 16.74|2679.0|
|    한국앤컴퍼니|  9.35|1749.0|
|      삼천당제약|293.58| 265.0|
+----------------+------+------+
only showing top 10 rows



In [190]:
# Spark DataFrame의 NaN을 Null로 변환하기. 
sdf = spark.createDataFrame([(1.0, None), (float('nan'), 'foo')], ("x", "y"))
print(sdf.show())

# nan을 null로 변경
sdf.replace(float('nan'), None).show()


# fillna(None) 은 오류 발생. 
sdf.fillna(value=None)

+---+----+
|  x|   y|
+---+----+
|1.0|null|
|NaN| foo|
+---+----+

None
+----+----+
|   x|   y|
+----+----+
| 1.0|null|
|null| foo|
+----+----+



PySparkTypeError: [NOT_BOOL_OR_DICT_OR_FLOAT_OR_INT_OR_STR] Argument `value` should be a bool, dict, float, int or str, got NoneType.

### 결손(Null) 데이터 처리하기
* DataFrame의 fillna() 메소드, 또는 DataFrameNaFunctions 객체인 DataFrame.na의 fill() 메소드를 이용
* DataFrame.fillna(value=값, subset=['컬럼1', 컬럼2])로 형태로 사용. value는 결측값에 입력될 값, subset은 대상 컬럼. subset을 지정하지 않으면 전체 컬럼에 적용. 
* subset을 지정하지 않고 value에 숫자값을 입력하면 숫자형 컬럼만 결손값을 처리함. 비슷하게 value에 문자값을 입력하면 문자형 컬럼만 결손값을 처리함.
* value는 반드시 단일 값이 들어가야함. 단일 값을 가지는 DataFrame은 안됨.

In [191]:
# 결측값을 per 변수의 평균값으로 대체
stock_pdf['per'] = stock_pdf['per'].fillna(stock_pdf['per'].mean(), inplace=False)

In [192]:
type(stock_sdf.na)

pyspark.sql.dataframe.DataFrameNaFunctions

In [193]:
print('subset을 지정하지 않고 숫자형 컬럼에 결측치 처리')
stock_sdf.fillna(value=999).show(3)
stock_sdf.na.fill(value=999).show(3)

print('subset을 지정하지 않고 문자형 컬럼에 결측치 처리')
stock_sdf.fillna(value='NA').show(3)

subset을 지정하지 않고 숫자형 컬럼에 결측치 처리
+--------+--------+----+------+------------+----------+-----+-----+-------+-------+-------+----+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+---------+------------+--------+--------+--------------+--------+
|종목코드|  종목명|종가|등락률|    시가총액|    기준일|  eps|  per|선행eps|선행per|    bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율|시가|고가|저가|거래량| 거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+--------+----+------+------------+----------+-----+-----+-------+-------+-------+----+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+---------+------------+--------+--------+--------------+--------+
|  000020|동화약품|9350| -0.85|261159244500|2024-03-15|736.0| 12.7|  999.0|  999.0|13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02|9450|9450|9300| 72206|674278570|      119195|   KOSPI|  보통주|      건강관리|  의약품|
|  000040|KR

In [195]:
print('per 컬럼 결측치 처리')
stock_sdf.fillna(value=999, subset=['per']).select('종목명', 'per', 'eps', '섹터').show(10) 

print('eps 컬럼 결측치 처리')
# titanic_pdf['Cabin'].fillna('NA', inplace=False)
stock_sdf.fillna(value=999, subset=['eps']).select('종목명', 'per', 'eps', '섹터').show(10) 


per 컬럼 결측치 처리
+--------------+-----+------+--------------+
|        종목명|  per|   eps|          섹터|
+--------------+-----+------+--------------+
|      동화약품| 12.7| 736.0|      건강관리|
|      KR모터스|999.0|  null|경기관련소비재|
|          경방|47.85| 177.0|경기관련소비재|
|    삼양홀딩스| 7.77|9173.0|          소재|
|  삼양홀딩스우|999.0|  null|          null|
|    하이트진로|16.24|1250.0|    필수소비재|
|하이트진로2우B|999.0|  null|          null|
|      유한양행|58.18|1272.0|      건강관리|
|    유한양행우|999.0|  null|          null|
|    CJ대한통운|15.21|8190.0|        산업재|
+--------------+-----+------+--------------+
only showing top 10 rows

eps 컬럼 결측치 처리
+--------------+-----+------+--------------+
|        종목명|  per|   eps|          섹터|
+--------------+-----+------+--------------+
|      동화약품| 12.7| 736.0|      건강관리|
|      KR모터스| null| 999.0|경기관련소비재|
|          경방|47.85| 177.0|경기관련소비재|
|    삼양홀딩스| 7.77|9173.0|          소재|
|  삼양홀딩스우| null| 999.0|          null|
|    하이트진로|16.24|1250.0|    필수소비재|
|하이트진로2우B| null| 999.0|          null|
|      유

In [197]:
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, col

# select avg(per) from stock_sdf
avg_per = stock_sdf.select(F.avg(F.col('per')))
print(avg_per.show())
print('### avg_per type:', type(avg_per))


+------------------+
|          avg(per)|
+------------------+
|51.476893750000045|
+------------------+

None
### avg_per type: <class 'pyspark.sql.dataframe.DataFrame'>


In [200]:
# 아래는 오류를 발생시킴. value 인자로 단일 값이 입력되어야 함. DataFrame은 입력 될 수 없음. 
stock_sdf.fillna(value=avg_per, subset=['per'])

PySparkTypeError: [NOT_BOOL_OR_DICT_OR_FLOAT_OR_INT_OR_STR] Argument `value` should be a bool, dict, float, int or str, got DataFrame.

In [198]:
# first()는 head()와 동일하게 동작. 하지만 first(N)은 존재하지 않으며 first()는 맨 처음 Row만 가져옴. 
avg_per_row = avg_per.first()
print(avg_per_row, type(avg_per_row))
print()

# 아래는 DataFrame의 단일 Row에서 맨 첫번째 개별 value를 가져옴. 
avg_per_value = avg_per.first()[0]
print(avg_per_value, type(avg_per_value))

Row(avg(per)=51.476893750000045) <class 'pyspark.sql.types.Row'>

51.476893750000045 <class 'float'>


In [199]:
stock_sdf.fillna(value=avg_per_value, subset=['per']).select('종목명', 'per', '섹터').show(10)

+--------------+------------------+--------------+
|        종목명|               per|          섹터|
+--------------+------------------+--------------+
|      동화약품|              12.7|      건강관리|
|      KR모터스|51.476893750000045|경기관련소비재|
|          경방|             47.85|경기관련소비재|
|    삼양홀딩스|              7.77|          소재|
|  삼양홀딩스우|51.476893750000045|          null|
|    하이트진로|             16.24|    필수소비재|
|하이트진로2우B|51.476893750000045|          null|
|      유한양행|             58.18|      건강관리|
|    유한양행우|51.476893750000045|          null|
|    CJ대한통운|             15.21|        산업재|
+--------------+------------------+--------------+
only showing top 10 rows



### 사용자 정의 함수(User Defined Function)을 DataFrame 가공 시 적용하는 법과 when 사용법
* UDF를 Spark DataFrame에 적용하려면 먼저 일반 함수를 만든 후에 이를 spark의 udf() 함수를 이용하여 DataFrame에서 사용할 수 있도록 변환해야 함. 
* pyspark.sql.functions의 when()은 SQL의 Case When Then... Else 구문과 동일하게 동작.

In [ ]:
import pyspark.sql.functions as F

avg_age = titanic_sdf.select(F.avg(F.col('Age')))
avg_age_row = avg_age.head()
avg_age_value = avg_age.head()[0]

# Spark DataFrame의 fillna()에 인자로 Dict를 입력하여 여러개의 컬럼들에 대해서 결측치 값을 입력할 수 있게 만들어줌. 
titanic_sdf_filled = titanic_sdf.fillna({'Age': avg_age_value, 
                                         'Cabin': 'C000',
                                         'Embarked': 'S'
})

In [ ]:
titanic_sdf_filled.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| C000|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| C000|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

In [ ]:
# 일반 python용 UDF를 작성. 반드시 입력 값과 반환 값을 설정
def get_category(age):
    cat = ''
    
    # age 값이 None일 경우는 NA를 Return
    #if age == None:
        #return 'NA'
    
    if age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else : cat = 'Elderly'
    
    return cat

In [ ]:
# pandas DataFrame에서 apply lambda 식으로 데이터 가공하기. age 값이 None/NaN 일 경우에도 else 조건에 의해 Elderly로 변환. 
titanic_pdf['Age_category'] = titanic_pdf['Age'].apply(lambda x: get_category(x))
titanic_pdf.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_category
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,Student
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,Young Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Young Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,Young Adult
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q,Elderly
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Adult
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S,Baby
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,None,S,Young Adult
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,None,C,Teenager


In [ ]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

# 일반 python용 UDF를 pyspark용 UDF로 변환. udf(lambda 입력변수: 일반 UDF, 해당 일반 UDF의 반환형)
udf_get_category = udf(lambda x:get_category(x), StringType() )

In [ ]:
# udf_get_category()에 Age 컬럼값을 입력하여 반환되는 값으로 새로운 컬럼 Age_Category를 생성
titanic_sdf_filled_01 = titanic_sdf_filled.withColumn("Age_Category",udf_get_category(col("Age")))
titanic_sdf_filled_01.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_Category|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S| Young Adult|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null

In [ ]:
from pyspark.sql.functions import when
                   
titanic_sdf_filled_02 = titanic_sdf_filled.withColumn('Age_category', when(F.col('Age') <= 5, 'Baby')
                                                                      .when(F.col('Age') <= 12, 'Child')
                                                                      .when(F.col('Age') <= 18, 'Teenage')
                                                                      .when(F.col('Age') <= 25, 'Student')
                                                                      .when(F.col('Age') <= 35, 'Young Adult')
                                                                      .when(F.col('Age') <= 60, 'Adult')
                                                                      .when(F.col('Age').isNull(), 'NA')
                                                                      .otherwise('Elderly'))

titanic_sdf_filled_02.limit(10).show()

''' Select a.*, CASE WHEN age <=6 THEN 'Baby'
                 WHEN age <=12 Then 'Child'
                 WHEN age <= 18 THEN 'Teenage'
                 WHEN age <= 25 THEN 'Student'
                 WHEN age <=35 THEN 'Young Adult'
                 WHEN age <=60 THEN 'Adult'
                 WHEN age is Null THEN 'NA'
                 ELSE 'Elderly' END from titanic_sdf a;
'''

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Age_category|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|     Student|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|       Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Young Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S| Young Adult|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null

In [ ]:
from pyspark.sql.functions import expr, col

titanic_sdf_filled_03 = titanic_sdf.withColumn('Age_category', expr("CASE WHEN age = 12 THEN 'Child' " + 
                                               " WHEN Age <= 18 THEN 'Teenage' " +
                                               " WHEN Age <= 25 THEN 'Student' " +
                                               " WHEN Age <= 35 THEN 'Young Adult' " + 
                                               " WHEN Age <= 60 THEN 'Adult' " + 
                                               " WHEN Age IS NULL THEN 'NA' " +
                                               " ELSE 'Elderly' "))
titanic_sdf_filled_03.limit(10).show()

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
<command-2719872723043665> in <module>
      1 from pyspark.sql.functions import expr, col
      2 
----> 3 titanic_sdf_filled_03 = titanic_sdf.withColumn('Age_category', expr("CASE WHEN age = 12 THEN 'Child' " + \
      4                                                " WHEN Age <= 18 THEN 'Teenage' " +
      5                                                " WHEN Age <= 25 THEN 'Student' " +

/databricks/spark/python/pyspark/sql/functions.py in expr(str)
   1457     """
   1458     sc = SparkContext._active_spark_context
-> 1459     return Column(sc._jvm.functions.expr(str))
   1460 
   1461 

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             ans

In [ ]:
f3 = df.withColumn("new_gender", expr("CASE WHEN gender = 'M' THEN 'Male' " + 
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
               "ELSE gender END"))